In [ ]:
!pip install -q datasets transformers transformers[torch] huggingface_hub accelerate

In [ ]:
from datasets import load_dataset

In [ ]:
emotion_dataset = load_dataset(path = "dair-ai/emotion")

README.md: 0.00B [00:00, ?B/s]

split/train-00000-of-00001.parquet:   0%|          | 0.00/1.03M [00:00<?, ?B/s]

split/validation-00000-of-00001.parquet:   0%|          | 0.00/127k [00:00<?, ?B/s]

split/test-00000-of-00001.parquet:   0%|          | 0.00/129k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/16000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [ ]:
emotion_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
})

In [ ]:
emotion_dataset['train']

Dataset({
    features: ['text', 'label'],
    num_rows: 16000
})

In [ ]:
emotion_dataset['train'][0]

{'text': 'i didnt feel humiliated', 'label': 0}

In [ ]:
emotion_dataset.set_format('pandas')

In [ ]:
emotion_df = emotion_dataset['train'][:]
emotion_df.head()

,text,label
0,i didnt feel humiliated,0
1,i can go from feeling so hopeless to so damned...,0
2,im grabbing a minute to post i feel greedy wrong,3
3,i am ever feeling nostalgic about the fireplac...,2
4,i am feeling grouchy,3


In [ ]:
classes = emotion_dataset['train'].features['label'].names
classes

['sadness', 'joy', 'love', 'anger', 'fear', 'surprise']

In [ ]:
emotion_df['label_name'] = emotion_df['label'].apply(lambda x: classes[x])
emotion_df.head()

,text,label,label_name
0,i didnt feel humiliated,0,sadness
1,i can go from feeling so hopeless to so damned...,0,sadness
2,im grabbing a minute to post i feel greedy wrong,3,anger
3,i am ever feeling nostalgic about the fireplac...,2,love
4,i am feeling grouchy,3,anger


In [ ]:
emotion_df['Words_per_emotion'] = emotion_df['text'].str.split().apply(len)
emotion_df.head()

,text,label,label_name,Words_per_emotion
0,i didnt feel humiliated,0,sadness,4
1,i can go from feeling so hopeless to so damned...,0,sadness,21
2,im grabbing a minute to post i feel greedy wrong,3,anger,10
3,i am ever feeling nostalgic about the fireplac...,2,love,18
4,i am feeling grouchy,3,anger,4


In [ ]:
emotion_dataset.reset_format()

Load the tokenizer

In [ ]:
from transformers import AutoTokenizer

In [ ]:
model_ckpt = "distilbert/distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [ ]:
text = "Hello! Welcome to my world."
encoded_text = tokenizer(text = text)
encoded_text

{'input_ids': [101, 7592, 999, 6160, 2000, 2026, 2088, 1012, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
tokens = tokenizer.convert_ids_to_tokens(encoded_text['input_ids'])
print(tokens)

['[CLS]', 'hello', '!', 'welcome', 'to', 'my', 'world', '.', '[SEP]']


In [ ]:
tokenizer.vocab_size, tokenizer.model_max_length

(30522, 512)

In [ ]:
# Use the map method

def tokenize(batch):
  temp = tokenizer(text = batch['text'], padding=True, truncation=True)
  return temp

emotion_encoded = emotion_dataset.map(tokenize, batched=True, batch_size=2000)

Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [ ]:
emotion_encoded

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 2000
    })
})

In [ ]:
print(emotion_encoded['train'][0])

{'text': 'i didnt feel humiliated', 'label': 0, 'input_ids': [101, 1045, 2134, 2102, 2514, 26608, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}


### **Loading the AutoModel**


In [ ]:
from transformers import AutoModel

model = AutoModel.from_pretrained(model_ckpt)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [ ]:
text_input = "Hello! Welcome to my world."

text_input_encoded = tokenizer(text=text_input, return_tensors="pt")
text_input_encoded

{'input_ids': tensor([[ 101, 7592,  999, 6160, 2000, 2026, 2088, 1012,  102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [ ]:
import torch

In [ ]:
with torch.inference_mode():
  output = model(**text_input_encoded)

output

BaseModelOutput(last_hidden_state=tensor([[[-0.1771, -0.0132,  0.1112,  ..., -0.0557,  0.2214,  0.3330],
         [-0.1610,  0.0799,  0.8072,  ..., -0.0745,  0.4777,  0.0154],
         [-0.2932,  0.1581,  0.6711,  ..., -0.1247,  0.4931,  0.6353],
         ...,
         [ 0.1061,  0.2477,  0.5413,  ..., -0.0231,  0.3397,  0.4355],
         [-0.3497, -0.8133,  0.2876,  ...,  0.6198,  0.1382, -0.3950],
         [ 0.9264,  0.1975, -0.2092,  ...,  0.2900, -0.4663, -0.1757]]]), hidden_states=None, attentions=None)

In [ ]:
last_hidden_state = output['last_hidden_state']
last_hidden_state

tensor([[[-0.1771, -0.0132,  0.1112,  ..., -0.0557,  0.2214,  0.3330],
         [-0.1610,  0.0799,  0.8072,  ..., -0.0745,  0.4777,  0.0154],
         [-0.2932,  0.1581,  0.6711,  ..., -0.1247,  0.4931,  0.6353],
         ...,
         [ 0.1061,  0.2477,  0.5413,  ..., -0.0231,  0.3397,  0.4355],
         [-0.3497, -0.8133,  0.2876,  ...,  0.6198,  0.1382, -0.3950],
         [ 0.9264,  0.1975, -0.2092,  ...,  0.2900, -0.4663, -0.1757]]])

### Loading the AutoModelForSequenceClassification

In [ ]:
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments
from transformers import Trainer

In [ ]:
num_labels = len(classes)
num_labels

6

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [ ]:
seq_model = AutoModelForSequenceClassification.from_pretrained(model_ckpt, num_labels = num_labels)
seq_model = seq_model.to(device)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model_name = "distill-bert-classifier"
batch_size = 64

training_args = TrainingArguments(
  output_dir = model_name,
  learning_rate = 2e-5,
  num_train_epochs = 2,
  weight_decay = 0.01,
  eval_strategy = 'epoch',
  disable_tqdm = False,
  per_device_train_batch_size = batch_size,
  per_device_eval_batch_size = batch_size,
  report_to = "none"
)

In [ ]:
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(test_data):
  labels = test_data.label_ids
  preds = test_data.predictions.argmax(-1)
  accuracy = accuracy_score(labels, preds)
  f1 = f1_score(labels, preds, average = "weighted")
  return {"accuracy": accuracy, "f1": f1}

In [ ]:
trainer = Trainer(
  model = seq_model,
  args = training_args,
  train_dataset = emotion_encoded['train'],
  eval_dataset = emotion_encoded['validation'],
  processing_class = tokenizer,
  compute_metrics = compute_metrics,
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.300494,0.911000,0.910205
2,0.521600,0.212507,0.922500,0.922319


TrainOutput(global_step=500, training_loss=0.5216220092773437, metrics={'train_runtime': 234.9769, 'train_samples_per_second': 136.184, 'train_steps_per_second': 2.128, 'total_flos': 720342861696000.0, 'train_loss': 0.5216220092773437, 'epoch': 2.0})

In [ ]:
seq_model

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


In [ ]:
pred_outputs = trainer.predict(emotion_encoded['test'])

In [ ]:
pred_outputs.predictions

array([[ 4.426706  , -0.67501515, -1.2786628 , -0.28560215, -1.1436522 ,
        -1.689533  ],
       [ 4.542443  , -0.8973583 , -0.935464  , -0.7585854 , -1.0331234 ,
        -1.5624597 ],
       [ 4.547793  , -1.1093897 , -0.91364706, -0.94719106, -0.71510804,
        -1.5036792 ],
       ...,
       [-0.9084334 ,  4.6592927 , -0.30152324, -1.3261203 , -1.4603691 ,
        -0.92886513],
       [-0.95242953,  4.6266885 , -0.54498273, -1.2574588 , -1.1728404 ,
        -0.9419452 ],
       [-1.0140703 , -1.0275011 , -1.3883997 , -1.2594126 ,  2.4367788 ,
         2.1859658 ]], dtype=float32)

In [ ]:
pred_outputs.metrics

{'test_loss': 0.2117784172296524,
 'test_accuracy': 0.9195,
 'test_f1': 0.918908770266933,
 'test_runtime': 3.4148,
 'test_samples_per_second': 585.688,
 'test_steps_per_second': 9.371}

In [ ]:
import numpy as np
y_preds = np.argmax(pred_outputs.predictions, axis = -1)
y_true = emotion_encoded['test'][:]['label']

In [ ]:
from sklearn.metrics import classification_report
print(classes)
print(classification_report(y_true, y_preds))

['sadness', 'joy', 'love', 'anger', 'fear', 'surprise']
              precision    recall  f1-score   support

           0       0.95      0.96      0.96       581
           1       0.94      0.94      0.94       695
           2       0.78      0.82      0.80       159
           3       0.94      0.92      0.93       275
           4       0.88      0.90      0.89       224
           5       0.82      0.62      0.71        66

    accuracy                           0.92      2000
   macro avg       0.88      0.86      0.87      2000
weighted avg       0.92      0.92      0.92      2000



Test the Model

In [ ]:
text = "The weather is very bad today!"
encoded_text = tokenizer(text=text, return_tensors="pt").to(device)

with torch.inference_mode():
  output = seq_model(**encoded_text)

logits = output.logits
pred = torch.argmax(logits, dim = -1).item()
pred, classes[pred]

(0, 'sadness')

Saving the Model and Tokenizer locally

In [ ]:
seq_model.save_pretrained('./emotion_classifier')
tokenizer.save_pretrained("./emotion_classifier")

('./emotion_classifier/tokenizer_config.json',
 './emotion_classifier/special_tokens_map.json',
 './emotion_classifier/vocab.txt',
 './emotion_classifier/added_tokens.json',
 './emotion_classifier/tokenizer.json')

Loading the Locally saved Model and Tokenizer

In [ ]:
loaded_model = AutoModelForSequenceClassification.from_pretrained("./emotion_classifier")
loaded_tokenizer = AutoTokenizer.from_pretrained('./emotion_classifier')

In [ ]:
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
loaded_model = loaded_model.to(device)

In [ ]:
text = "The weather is bad today."
encoded_text = loaded_tokenizer(text = text, return_tensors="pt").to(device)

with torch.inference_mode():
  outputs = loaded_model(**encoded_text)

logits = outputs.logits
pred = torch.argmax(logits, dim = -1).item()
pred, classes[pred]

(0, 'sadness')

Pushing the model, tokenizer to Huggingface

In [ ]:
from huggingface_hub import HfApi, notebook_login

In [ ]:
notebook_login()

In [ ]:
loaded_model.push_to_hub(repo_id="tsid7710/distillbert-emotion-model")
loaded_tokenizer.push_to_hub(repo_id="tsid7710/distillbert-emotion-model")

README.md: 0.00B [00:00, ?B/s]

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...r9wvzv8/model.safetensors:   0%|          |  575kB /  268MB            

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/tsid7710/distillbert-emotion-model/commit/219374bd403fe0866a57adbc49b5ac59d5e9044d', commit_message='Upload tokenizer', commit_description='', oid='219374bd403fe0866a57adbc49b5ac59d5e9044d', pr_url=None, repo_url=RepoUrl('https://huggingface.co/tsid7710/distillbert-emotion-model', endpoint='https://huggingface.co', repo_type='model', repo_id='tsid7710/distillbert-emotion-model'), pr_revision=None, pr_num=None)

**Loading the tokenizer and Model from Huggingface**

In [ ]:
hf_tokenizer = AutoTokenizer.from_pretrained("tsid7710/distillbert-emotion-model")

hf_model = AutoModelForSequenceClassification.from_pretrained("tsid7710/distillbert-emotion-model")
hf_model = hf_model.to(device)

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/831 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [ ]:
text = "The weather is bad today."
encoded_text = hf_tokenizer(text = text, return_tensors="pt").to(device)

with torch.inference_mode():
  outputs = hf_model(**encoded_text)

logits = outputs.logits
pred = torch.argmax(logits, dim = -1).item()
pred, classes[pred]

(0, 'sadness')